In [175]:
import mysql.connector
import pandas as pd
import numpy as np

In [176]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [177]:
mydb = mysql.connector.connect(
    host="localhost",
    user="sungkyun",
    password="chung741",
    database="sungkyun"
)

mycursor = mydb.cursor()

sql = 'SELECT DISTINCT built_years, size, street_adress_code, dong_code, max(floor) FROM apart_price_clone GROUP BY built_years, size, street_adress_code, dong_code'
df = pd.read_sql(sql, mydb)
df.to_csv('apart_learning_data.csv')

In [184]:
df

,built_years,size,street_adress_code,dong_code,max(floor)
0,1961,26.5400,414821100042,1153010200,4
1,1961,35.5500,414830400011,1153010200,5
2,1961,44.5500,414820200027,1153010200,5
3,1961,48.1300,414831700011,1153010200,5
4,1961,48.9600,414820400062,1153010200,1
...,...,...,...,...,...
35009,2021,84.9500,414238000014,1147010100,6
35010,2021,84.9712,312400400055,1174010300,18
35011,2021,84.9800,311100700220,1138010700,15
35012,2021,84.9800,415450900017,1156013200,22


In [178]:
sql = 'SELECT street_adress_code, size FROM apart_price_clone GROUP BY street_adress_code, size HAVING COUNT(*) > 30'
df_sold = pd.read_sql(sql, mydb)

In [ ]:
sold_prices = np.zeros((7263, 30))
last_sold_price = np.zeros((7263,1))

adress_codes = df_sold['street_adress_code'].array
sizes = df_sold['size'].array
for i in range(len(sizes)):
    sql = 'SELECT price FROM apart_price_clone WHERE street_adress_code=' + str(adress_codes[i]) + ' AND size=' + str(sizes[i]) + ' ORDER BY sold_date DESC LIMIT 31'
    price_data = pd.read_sql(sql, mydb)
    data = np.flip(price_data['price'].to_numpy())
    sold_prices[i] = data[:30]
    last_sold_price[i] = data[30:]
        
np.save('sold_prices.npy', sold_prices)
np.save('last_sold_price.npy', last_sold_price)

In [179]:
sold_prices = np.load('sold_prices.npy')
last_sold_price = np.load('last_sold_price.npy')

In [181]:
df_sold['prices'] = sold_prices.tolist()
df_sold['labels'] = last_sold_price

In [186]:
df_sold

,street_adress_code,size,prices,labels
0,200000300569,66.90,"[22500.0, 22500.0, 21000.0, 22800.0, 21800.0, ...",38000.0
1,200000300569,84.29,"[27500.0, 24000.0, 22500.0, 23650.0, 25000.0, ...",41000.0
2,200000300575,84.96,"[25500.0, 25500.0, 24900.0, 26000.0, 24000.0, ...",43500.0
3,200000300775,59.86,"[42000.0, 40800.0, 41700.0, 40600.0, 43000.0, ...",71000.0
4,200000300775,71.86,"[44700.0, 43800.0, 43200.0, 45000.0, 46000.0, ...",90500.0
...,...,...,...,...
7258,417242800012,84.91,"[31000.0, 33800.0, 33700.0, 30000.0, 34000.0, ...",66500.0
7259,417242800012,93.55,"[28200.0, 28200.0, 31500.0, 34500.0, 35500.0, ...",75000.0
7260,417243500034,59.19,"[29700.0, 30500.0, 30000.0, 30900.0, 30000.0, ...",81400.0
7261,417243500034,69.17,"[36000.0, 35900.0, 37500.0, 38000.0, 36400.0, ...",103000.0


In [182]:
merged_df = df_sold.merge(df, on=['street_adress_code','size'], how='left')
merged_df = pd.concat([merged_df.prices.apply(pd.Series), merged_df.drop('prices', axis=1)], axis=1)

In [183]:
merged_df

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,street_adress_code,size,labels,built_years,dong_code,max(floor)
0,22500.0,22500.0,21000.0,22800.0,21800.0,24000.0,21400.0,24700.0,23500.0,24500.0,...,33000.0,34900.0,35500.0,35700.0,200000300569,66.90,38000.0,2002,1147010300,14
1,27500.0,24000.0,22500.0,23650.0,25000.0,22800.0,25000.0,23000.0,22900.0,22500.0,...,32000.0,37700.0,38900.0,39300.0,200000300569,84.29,41000.0,2002,1147010300,14
2,25500.0,25500.0,24900.0,26000.0,24000.0,23000.0,24300.0,25500.0,26000.0,28800.0,...,38000.0,37000.0,43500.0,43500.0,200000300575,84.96,43500.0,2002,1147010300,14
3,42000.0,40800.0,41700.0,40600.0,43000.0,44000.0,42500.0,43400.0,46400.0,45800.0,...,59500.0,60000.0,60000.0,71000.0,200000300775,59.86,71000.0,2014,1153010700,24
4,44700.0,43800.0,43200.0,45000.0,46000.0,45200.0,46800.0,45000.0,42870.0,46500.0,...,72000.0,83000.0,81000.0,87000.0,200000300775,71.86,90500.0,2014,1153010700,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7287,31000.0,33800.0,33700.0,30000.0,34000.0,36400.0,36300.0,28500.0,35500.0,38000.0,...,61500.0,62500.0,61000.0,67600.0,417242800012,84.91,66500.0,1997,1174010800,16
7288,28200.0,28200.0,31500.0,34500.0,35500.0,34800.0,38600.0,38000.0,36500.0,38300.0,...,58500.0,69000.0,69900.0,67300.0,417242800012,93.55,75000.0,1997,1174010800,16
7289,29700.0,30500.0,30000.0,30900.0,30000.0,28500.0,28700.0,27800.0,25750.0,26000.0,...,50500.0,58500.0,75000.0,76000.0,417243500034,59.19,81400.0,1988,1174010600,10
7290,36000.0,35900.0,37500.0,38000.0,36400.0,36700.0,37300.0,37100.0,36800.0,38000.0,...,77000.0,77000.0,89700.0,94000.0,417243500034,69.17,103000.0,1988,1174010600,12


In [188]:
features = merged_df.loc[:, merged_df.columns != 'labels']
labels = merged_df['labels']
feat_train, feat_test, lab_train, lab_test = train_test_split(features, labels, test_size=0.2)
model = RandomForestRegressor(random_state=2)
model.fit(feat_train, lab_train)
print(model.score(feat_test, lab_test))
print(model.feature_importances_)

0.9633700124841137
[7.13806097e-04 5.73861896e-04 5.23835209e-04 4.83838489e-04
 5.51112965e-04 4.88951629e-04 4.53554831e-04 3.94888046e-04
 4.45441238e-04 3.82959259e-04 3.71972680e-04 4.41612540e-04
 5.04008323e-04 3.83294222e-04 5.35733168e-04 4.57965231e-04
 4.74656794e-04 4.74944089e-04 5.76796052e-04 6.84186035e-04
 8.23200854e-04 1.68573210e-03 1.74639998e-03 1.03572829e-03
 1.15076821e-03 3.10129261e-03 4.06516025e-03 6.29261220e-03
 2.20217719e-02 9.41324606e-01 1.29821679e-03 2.24194727e-03
 1.24873971e-03 9.27347173e-04 1.11905815e-03]


In [190]:
df_predict = merged_df.loc[:, merged_df.columns != 0]
cl = df_predict.columns.tolist()
cl = cl[:29] + [cl[31]] + cl[29:31] + cl[32:]
df_predict = df_predict[cl]
df_predict

,1,2,3,4,5,6,7,8,9,10,...,26,27,28,29,labels,street_adress_code,size,built_years,dong_code,max(floor)
0,22500.0,21000.0,22800.0,21800.0,24000.0,21400.0,24700.0,23500.0,24500.0,24800.0,...,33000.0,34900.0,35500.0,35700.0,38000.0,200000300569,66.90,2002,1147010300,14
1,24000.0,22500.0,23650.0,25000.0,22800.0,25000.0,23000.0,22900.0,22500.0,24300.0,...,32000.0,37700.0,38900.0,39300.0,41000.0,200000300569,84.29,2002,1147010300,14
2,25500.0,24900.0,26000.0,24000.0,23000.0,24300.0,25500.0,26000.0,28800.0,28400.0,...,38000.0,37000.0,43500.0,43500.0,43500.0,200000300575,84.96,2002,1147010300,14
3,40800.0,41700.0,40600.0,43000.0,44000.0,42500.0,43400.0,46400.0,45800.0,46000.0,...,59500.0,60000.0,60000.0,71000.0,71000.0,200000300775,59.86,2014,1153010700,24
4,43800.0,43200.0,45000.0,46000.0,45200.0,46800.0,45000.0,42870.0,46500.0,44000.0,...,72000.0,83000.0,81000.0,87000.0,90500.0,200000300775,71.86,2014,1153010700,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7287,33800.0,33700.0,30000.0,34000.0,36400.0,36300.0,28500.0,35500.0,38000.0,40500.0,...,61500.0,62500.0,61000.0,67600.0,66500.0,417242800012,84.91,1997,1174010800,16
7288,28200.0,31500.0,34500.0,35500.0,34800.0,38600.0,38000.0,36500.0,38300.0,37750.0,...,58500.0,69000.0,69900.0,67300.0,75000.0,417242800012,93.55,1997,1174010800,16
7289,30500.0,30000.0,30900.0,30000.0,28500.0,28700.0,27800.0,25750.0,26000.0,28000.0,...,50500.0,58500.0,75000.0,76000.0,81400.0,417243500034,59.19,1988,1174010600,10
7290,35900.0,37500.0,38000.0,36400.0,36700.0,37300.0,37100.0,36800.0,38000.0,37500.0,...,77000.0,77000.0,89700.0,94000.0,103000.0,417243500034,69.17,1988,1174010600,12


In [194]:
predicted = merged_df.loc[:, (merged_df.columns == 'street_adress_code') | (merged_df.columns == 'size')]
predicted['prediction'] = model.predict(df_predict)
predicted.to_csv("predictions.csv")

/Users/chungsungkyun/opt/anaconda3/envs/cse163/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
